In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 9bee7037-1775-4339-856e-c1a80a051f39
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session 9bee7037-1775-4339-856e-c1a80a051f39 to get into ready status...
Session 9bee7037-1775-4339-856e-c1a80a051f39 ha

In [1]:
import requests
import csv
import boto3
import json
from io import StringIO

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Trying to create a Glue session for the kernel.
Session Type: glueetl
Session ID: 826bd454-e3c5-4287-8d31-78e618a1a104
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session 826bd454-e3c5-4287-8d31-78e618a1a104 to get into ready status...
Session 826bd454-e3c5-4287-8d31-78e618a1a104 has been created.



In [6]:
import requests
import json
import csv

# API 키 및 URL 설정
key = '0jt_b24c2c_3p_c0j2t3t_231e3p23o_'
url = f'https://open.jejudatahub.net/api/proxy/ttaaaa2D88t2a217D9b2a1a1b1D337D1/{key}'

# CSV 저장 파일명
csv_file = 'seogwipo_bike_storage.csv'

# 데이터 저장을 위한 리스트
all_data = []

# number 값에 따라 데이터를 불러옴
for number in [1, 2]:
    # API 요청 파라미터
    params = {
        'number': number,
        'limit': 100
    }

    # API 요청
    response = requests.get(url, params=params, timeout=10)

    if response.status_code == 200:
        contexts = response.text
        
        # JSON 데이터 파싱
        data = json.loads(contexts)
        all_data.extend(data.get('data', []))
    else:
        print(f"API 요청 실패 (number={number}): {response.status_code}")
        print("응답 내용:", response.text)



In [10]:
print(all_data)

[{'name': '남원읍사무소 주차장 옆', 'addressDoro': '제주특별자치도 서귀포시 남원읍 태위로 695', 'addressJibeon': '제주특별자치도 서귀포시 남원읍 남원리 205-1', 'latitude': '33.27987032', 'longitude': '126.7206485', 'storeNumber': '10', 'installationYear': '2000', 'airInjectorFlag': 'Y', 'repairDeskFlag': 'N', 'phoneNumber': '064-760-3094', 'managementAgency': '남원읍', 'registrationDate': '2022-12-15'}, {'name': '하수해수욕장 동쪽주차장', 'addressJibeon': '제주특별자치도 서귀포시 대정읍 하모리 276', 'latitude': '33.21082586', 'longitude': '126.2630076', 'storeNumber': '20', 'installationYear': '2012', 'airInjectorFlag': 'Y', 'repairDeskFlag': 'N', 'phoneNumber': '064-760-3094', 'managementAgency': '하수해수욕장', 'registrationDate': '2022-12-15'}, {'name': '덕수리 조각공원 주차장', 'addressJibeon': '제주특별자치도 서귀포시 안덕면 덕수리 613-2', 'latitude': '33.2511814', 'longitude': '126.3231603', 'storeNumber': '10', 'installationYear': '2009', 'airInjectorFlag': 'Y', 'repairDeskFlag': 'N', 'phoneNumber': '064-760-3094', 'managementAgency': '덕수리 조각공원', 'registrationDate': '2022-12-15'}, {'n

In [38]:
import boto3
import json
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Spark 세션 생성
spark = SparkSession.builder.appName('S3_Save_Data').getOrCreate()

# S3 버킷 및 경로 설정
bucket_name = 'airflow-test001'
s3_output_path = f"s3://{bucket_name}/data/seogwipo_bike_storage_info/"


# Pandas DataFrame으로 변환
df = pd.DataFrame(all_data)

# print(df)
# df.columns


In [39]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, DateType

# 스키마 정의
schema = StructType([
    StructField("name", StringType(), True),
    StructField("addressDoro", StringType(), True),
    StructField("addressJibeon", StringType(), True),
    StructField("latitude", StringType(), True),  
    StructField("longitude", StringType(), True), 
    StructField("storeNumber", StringType(), True),  
    StructField("installationYear", StringType(), True),  # 정수형으로 지정
    StructField("airInjectorFlag", StringType(), True),  # 'Y', 'N' 값 (문자열로 두되, 후에 Boolean으로 변환 가능)
    StructField("repairDeskFlag", StringType(), True),  # 'Y', 'N' 값 (문자열로 두되, 후에 Boolean으로 변환 가능)
    StructField("phoneNumber", StringType(), True),
    StructField("managementAgency", StringType(), True),
    StructField("registrationDate", StringType(), True), 
    StructField("installationForm", StringType(), True),
    StructField("awningFlag", StringType(), True),  # 'Y', 'N' 값 (문자열로 두되, 후에 Boolean으로 변환 가능)
    StructField("airInjectorType", StringType(), True)
])

# Spark DataFrame으로 변환
spark_df = spark.createDataFrame(df, schema)


/opt/amazon/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


In [40]:
# 데이터를 하나의 파일로 저장 (coalesce(1)로 파티션 수를 1로 변경)
spark_df.coalesce(1).write.option("header", "true").csv(s3_output_path)
print(f"Data successfully written to {s3_output_path}")

Data successfully written to s3://airflow-test001/data/seogwipo_bike_storage_info/
